
## 🥇 Camada Gold – Analises e Consultas


#### 🪄 Passo 1 : Consultas (Pedida No Desafio + Extra) + Criação da Tabela

###### 💾 Analise 1: Quantidade de Transações por mês por usuário

In [0]:
%sql
CREATE OR REPLACE TABLE adb_cliente_savana_prd.daniel_gold.1_qtd_transacoes_mes_usuario AS
SELECT 
    ID_Cliente,
    Nome,
    CPF,
    MONTH(DataCriacao) AS Mes,
    YEAR(DataCriacao) AS Ano,
    COUNT(ID_Transacao) AS Qtd_Transacoes
FROM adb_cliente_savana_prd.daniel_silver.baseclientes_transacoes_teste
GROUP BY ID_Cliente, YEAR(DataCriacao), MONTH(DataCriacao), Nome, CPF
ORDER BY ID_Cliente, Mes, Ano


###### 💾 Analise 2: Valor total de transação por usuário por mês

In [0]:
%sql
CREATE OR REPLACE TABLE adb_cliente_savana_prd.daniel_gold.2_valor_total_transacoes_mes_usuario AS
SELECT 
    ID_Cliente,
    Nome,
    CPF,
    MONTH(DataCriacao) AS Mes,
    YEAR(DataCriacao) AS Ano,
    SUM(Valor) AS Valor_Total
FROM adb_cliente_savana_prd.daniel_silver.baseclientes_transacoes_teste
GROUP BY ID_Cliente, Nome, CPF, YEAR(DataCriacao), MONTH(DataCriacao)
ORDER BY ID_Cliente, Ano, Mes

###### 💾 Analise 3: Volume de transações por cidade

In [0]:
%sql
CREATE OR REPLACE TABLE adb_cliente_savana_prd.daniel_gold.3_volume_transacoes_por_cidade AS
SELECT 
    Cidade,
    COUNT(*) AS Qtd_Transacoes,
    ROUND(SUM(Valor), 4) AS Volume_Total
FROM adb_cliente_savana_prd.daniel_silver.baseclientes_transacoes_teste
GROUP BY Cidade
ORDER BY Volume_Total DESC

###### 💾 Analise 4: Última transação realizada por cada cliente

In [0]:
%sql
CREATE OR REPLACE TABLE adb_cliente_savana_prd.daniel_gold.4_ultima_transacao_cliente AS
WITH top_transacoes AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY ID_Cliente
           ORDER BY DataCriacao DESC
         ) AS rn
  FROM adb_cliente_savana_prd.daniel_silver.baseclientes_transacoes_teste
)
SELECT 
  ID_Cliente,
  Nome,
  CPF,
  Cidade,
  DataCriacao,
  Valor,
  Tipo,
  Meio_de_Pgmto
FROM top_transacoes
WHERE rn = 1


###### 💾 Analise 5: Quantidade de registros com Data Futura x Data Anterior ou igual ao dia atual (Extra)

In [0]:
%sql
CREATE OR REPLACE TABLE adb_cliente_savana_prd.daniel_gold.5_distribuicao_clientes_cidade AS
SELECT 
    Cidade,
    COUNT(DISTINCT ID_Cliente) AS Qtd_Clientes
FROM adb_cliente_savana_prd.daniel_silver.baseclientes_transacoes_teste
GROUP BY Cidade
ORDER BY Qtd_Clientes DESC;

###### 💾 Analise 6: Quantidade de Registros em duas categorias: Data Futura e Data Válida (Extra)

In [0]:
%sql
CREATE OR REPLACE TABLE adb_cliente_savana_prd.daniel_gold.6_analise_datas_validas AS
SELECT 
  SUM(CASE WHEN DataCriacao > current_date() THEN 1 ELSE 0 END) AS Data_Futura,
  SUM(CASE WHEN DataCriacao <= current_date() THEN 1 ELSE 0 END) AS Data_Valida
FROM adb_cliente_savana_prd.daniel_silver.baseclientes_transacoes_teste